In [1]:
import numpy as np
from src.Data_fetching import get_pvgis_hourly
from src.Tensors import Tensors

In [2]:
# Data fetching
latitude =52
longitude = 0
optimal_angles = 1                                                                    # 1 if azimuth and tilt are not known, otherwise 0
azimuth = None                                                                        # None if azimuth is not known
tilt = None                                                                           # None if tilt is not known
peak_power = 2.48                                                                     # The peak output of the system
loss = 14                        

In [3]:
start_date = 2005
end_date = None

parameters_PVGIS = {'lat': latitude,
              'lon': longitude,
              'startyear': start_date,
              'endyear': end_date,
              'outputformat': 'json',
              'angle': tilt,
              'aspect': azimuth,
              'optimalangles': optimal_angles,
              'pvcalculation': 1,
              'components': 1,
              'peakpower': peak_power,
              'loss': loss,
              'localtime': 0}

In [4]:
pv_data = get_pvgis_hourly(parameters_PVGIS)[['P']]

In [5]:
pv_data.index = pv_data.index.floor('h').tz_localize(None)

In [6]:
pv_data.loc[:, 'hour_sin'] = np.sin(2 * np.pi * pv_data.index.hour / 24)
pv_data.loc[:, 'hour_cos'] = np.cos(2 * np.pi * pv_data.index.hour / 24)

In [7]:
pv_data

,P,hour_sin,hour_cos
time,,,
2005-01-01 00:00:00,0.0,0.000000,1.000000
2005-01-01 01:00:00,0.0,0.258819,0.965926
2005-01-01 02:00:00,0.0,0.500000,0.866025
2005-01-01 03:00:00,0.0,0.707107,0.707107
2005-01-01 04:00:00,0.0,0.866025,0.500000
...,...,...,...
2020-12-31 19:00:00,0.0,-0.965926,0.258819
2020-12-31 20:00:00,0.0,-0.866025,0.500000
2020-12-31 21:00:00,0.0,-0.707107,0.707107


In [8]:
past_features = ['P']                                                                 
future_features = ['hour_sin',                                                        
                   'hour_cos']

lags = 24                                                                             
forecast_period = 24                                                                  
gap = 0                                                                               
forecast_gap = 0                                                                      

In [9]:
# Get the data in the torch.tensor format
target_tensors = Tensors(pv_data,
                         'P',
                         past_features ,
                         future_features,
                         lags,
                         forecast_period,
                         gap=gap,
                         forecast_gap=forecast_gap,
                         train_test_split=0.8)

# Split the data into train and test sets with separate tensors for features (X) and the target (y)
X_train, X_test, y_train, y_test = target_tensors.create_tensor()
X_cal = X_train[-X_test.shape[0]:,:,:]
X_train = X_train[:-X_test.shape[0],:,:]
y_cal = y_train[-y_test.shape[0]:,:]
y_train = y_train[:-y_test.shape[0],:]

X_train.shape, X_cal.shape, X_test.shape, y_train.shape, y_cal.shape, y_test.shape

(torch.Size([3505, 24, 3]),
 torch.Size([1169, 24, 3]),
 torch.Size([1169, 24, 3]),
 torch.Size([3505, 24]),
 torch.Size([1169, 24]),
 torch.Size([1169, 24]))

In [10]:
from src.DeepAR import DeepAR

In [18]:
deep = DeepAR(3, 10, 1, 0)

In [19]:
deep

DeepAR(
  (lstm): LSTM(3, 10, batch_first=True)
  (linear): Linear(in_features=10, out_features=2, bias=True)
)

In [20]:
X_train[0:0,:,:].shape

torch.Size([0, 24, 3])

In [22]:
deep(X_train[0,:,:])

(tensor([-0.1417, -0.1064, -0.0856, -0.0722, -0.0631, -0.0573, -0.0546, -0.0548,
         -0.0578, -0.0630, -0.0713, -0.0794, -0.0885, -0.0978, -0.1058, -0.1130,
         -0.1195, -0.1248, -0.1284, -0.1298, -0.1289, -0.1255, -0.1196, -0.1115],
        grad_fn=<SelectBackward0>),
 tensor([0.5778, 0.5774, 0.5745, 0.5712, 0.5682, 0.5654, 0.5627, 0.5600, 0.5572,
         0.5544, 0.5517, 0.5488, 0.5462, 0.5440, 0.5422, 0.5410, 0.5406, 0.5409,
         0.5420, 0.5438, 0.5461, 0.5486, 0.5513, 0.5539],
        grad_fn=<AddBackward0>))